# Import libraries

In [ ]:
import uproot
import awkward as ak
import mplhep
%matplotlib inline 
#%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from brokenaxes import brokenaxes
import pandas as pd
from scipy.optimize import curve_fit

# Exercise 3: Measurement of the total production cross sections

Load Data

In [ ]:
meanenergy = np.array([88.47939, 89.46793, 90.22266, 91.22430, 91.96648, 92.96465, 93.71712])
lumi = np.array([463.9790, 667.5236, 486.7641, 2246.568, 535.9080, 450.6000, 709.6980])
stat = np.array([2.902361, 3.521166, 3.033955, 6.603405, 3.265110, 3.027953, 3.819882])
sys = np.array([3.104100, 4.471900, 3.261500, 15.04780, 3.585300, 3.020000, 4.762000])
all = np.array([4.249604, 5.691792, 4.454466, 16.43293, 4.849260, 4.276552, 6.104764])

In [ ]:
### Open file

file_Opal = uproot.open('daten_4.root')
ttree_name_Opal = 'myTTree'

### Print list of 'branches' of the TTree (i.e. list of variable names)
print('Opal',file_Opal[ttree_name_Opal].keys())

## Load branches
branches_Opal = file_Opal[ttree_name_Opal].arrays()

var_Pcharged = 'Pcharged'
pchar_Opal = ak.to_numpy(branches_Opal[var_Pcharged])

var_Ncharged = 'Ncharged'
nchar_Opal = ak.to_numpy(branches_Opal[var_Ncharged])

var_E_ecal = 'E_ecal'
E_ecal_Opal = ak.to_numpy(branches_Opal[var_E_ecal])

var_E_hcal = 'E_hcal'
E_hcal_Opal = ak.to_numpy(branches_Opal[var_E_hcal])

var_cos_thet = 'cos_thet'
cos_thet_Opal = ak.to_numpy(branches_Opal[var_cos_thet])

var_E_lep = 'E_lep'
E_lep_Opal = ak.to_numpy(branches_Opal[var_E_lep]) 

print(f"Opal {E_lep_Opal} min: {E_lep_Opal.min()}, max: {E_lep_Opal.max()}")

data_Opal = pd.DataFrame({'COM_energy':np.zeros(len(nchar_Opal)), 'ID': np.chararray(len(nchar_Opal)),'Ncharged': nchar_Opal, 'Pcharged': pchar_Opal, 'E_ecal': E_ecal_Opal, 'E_hcal': E_hcal_Opal, 'E_lep':E_lep_Opal, 'cos_thet': cos_thet_Opal})

### separate the 7 different energies

In [ ]:
def sevenEnergies(E_lep):
    Energies = np.array([])
    for i in range(len(E_lep)):
        if 2*E_lep[i]<(meanenergy[0]+meanenergy[1])/2:
            Energies = np.append(Energies,meanenergy[0])

        elif 2*E_lep[i]<(meanenergy[1]+meanenergy[2])/2:
            Energies = np.append(Energies,meanenergy[1])

        elif 2*E_lep[i]<(meanenergy[2]+meanenergy[3])/2:
            Energies = np.append(Energies,meanenergy[2])

        elif 2*E_lep[i]<(meanenergy[3]+meanenergy[4])/2:
            Energies = np.append(Energies,meanenergy[3])

        elif 2*E_lep[i]<(meanenergy[4]+meanenergy[5])/2:
            Energies = np.append(Energies,meanenergy[4])
        
        elif 2*E_lep[i]<(meanenergy[5]+meanenergy[6])/2:
            Energies = np.append(Energies,meanenergy[5])
        
        else:
            Energies = np.append(Energies,meanenergy[6])

    return Energies

In [ ]:
data_Opal.loc[:, ['COM_energy']] = sevenEnergies(data_Opal['E_lep'])

In [ ]:
plt.style.use(mplhep.style.ATLAS) # You can load ATLAS/CMS/ALICE plot style 
ratio = [1,4] # ratio of the two subplots

#Opal

bin_content_Opal, bin_edges_Opal, _ = plt.hist(data_Opal.loc[data_Opal['E_lep'] == 44.732]['E_ecal'],bins=50, histtype='step',  linewidth=2, edgecolor='b', hatch='/', label='E_cal Opal')
mid_Opal = 0.5*(bin_edges_Opal[1:] + bin_edges_Opal[:-1]) #Calculate midpoint of the bars

error_sizes = np.sqrt(bin_content_Opal)

plt.errorbar(mid_Opal, bin_content_Opal, yerr=error_sizes, fmt='none')


#plt.ylim(0, 0.7*10**4)  # most of the data



### Show the plot on screen
plt.xlim(0,150)
plt.legend(loc = 4)
plt.xlabel('E_ecal')
plt.ylabel('Number of events')
plt.show()

First determine the number of events in the handronic channel *and* in the three leptonic channels


In [ ]:
s_ratio= 0.470464817345979
sigma_ratio = 0.005072534012880613

In [ ]:
def classify_event_in_4CH(Ncharged, Pcharged, E_ecal, E_hcal, cos_thet):
    if len(Ncharged)==len(Pcharged)==len(E_ecal)==len(E_hcal):
        #print(Ncharged)
        PI = np.chararray((len(Ncharged),1), itemsize=2)[:]
        PI[:] = 'NC'      # create start PI array with all events unclassified
        NC = 0      
        ee = 0 
        es = 0     
        qq = 0
        mm = 0
        tt = 0
        #print(f'Classification for Energy {meanenergy[I]}:\n')                 
        for i in range(len(Ncharged)):
            
            if (Ncharged[i] >= 7) & (E_ecal[i]>20):
                PI[i] = "qq"
                qq += 1
            elif (E_ecal[i] >= 60): # electron s channel cut
                PI[i] = "ee"
                ee += 1
            elif (70<=Pcharged[i] <= 110) & (E_ecal[i] < 20):
                PI[i] = "mm"
                mm += 1
            elif (Pcharged[i] < 10) & (E_ecal[i] < 10):
                PI[i] = "mm"
                mm += 1
            elif (E_ecal[i] < 100) & (1 < Pcharged[i] < 75):
                PI[i] = "tt"
                tt += 1
            else:
                PI[i] = "NC"
                NC += 1
        L = len(Ncharged) * 0.470464817345979 # take into account that we only look at the S-channel hece 'Lenght' (total number of events are only number of all ee events times s-channel frac)
        for i in range(len(Ncharged)):
            if (PI[i] == b'ee') & (-0.9 < cos_thet[i] < 0.1): # electron s channel cut
                PI[i] = "es"
                es += 1 * 2.144772117962466 # correct for only selecting part of the S_channel
        #print(PI)
        print('ee, mm, tt, qq, NC,len(Ncharged)')
        print(ee, mm, tt, qq, NC,len(Ncharged))
        print()
    else:
        print('Unequaly long input arrays')
    return [[ee, mm, tt, qq, NC, len(Ncharged)], PI, [es/L, mm/len(Ncharged), tt/len(Ncharged), qq/len(Ncharged), NC/len(Ncharged)],[es, mm, tt, qq, NC, len(Ncharged)]]

### Add particle ID

In [ ]:
nchar = data_Opal['Ncharged'].to_numpy()
pchar = data_Opal['Pcharged'].to_numpy()
E_ecal = data_Opal['E_ecal'].to_numpy()
E_hcal = data_Opal['E_hcal'].to_numpy()
cos_thet = data_Opal['cos_thet'].to_numpy()
data_Opal.loc[:, ['ID']] = classify_event_in_4CH(nchar, pchar, E_ecal, E_hcal, cos_thet)[1]
#len(classify_event_in_4CH(nchar, pchar, E_ecal, E_hcal, i)[1])

### create callification Matrix sorted by energies

In [ ]:
classify_Opal = np.zeros((7,6))
for i in range(0,7,1):
    print(i)
    nchar = data_Opal.loc[data_Opal['COM_energy'] == meanenergy[i]]['Ncharged'].to_numpy()
    pchar = data_Opal.loc[data_Opal['COM_energy'] == meanenergy[i]]['Pcharged'].to_numpy()
    E_ecal = data_Opal.loc[data_Opal['COM_energy'] == meanenergy[i]]['E_ecal'].to_numpy()
    E_hcal = data_Opal.loc[data_Opal['COM_energy'] == meanenergy[i]]['E_hcal'].to_numpy()
    cos_thet = data_Opal.loc[data_Opal['COM_energy'] == meanenergy[i]]['cos_thet'].to_numpy()
    classify_Opal[i] = classify_event_in_4CH(nchar, pchar, E_ecal, E_hcal, cos_thet)[3]
classify_Opal_error = np.sqrt(classify_Opal)

In [ ]:
classify_Opal_df = pd.DataFrame(dict(zip(['es', 'mm', 'tt', 'qq', 'NC','length'], classify_Opal.transpose())))
classify_Opal_df['CMS_energies'] = meanenergy
classify_Opal_df = classify_Opal_df.join(pd.DataFrame(classify_Opal_error[:, :4], columns = ['es_u', 'mm_u', 'tt_u', 'qq_u']))
classify_Opal_df

In [ ]:
i = 0
plt.hist(data_Opal[(data_Opal['COM_energy'] == meanenergy[i]) & (data_Opal['ID'] == b'ee') & (data_Opal['cos_thet']<=0.25)]['E_ecal'].to_numpy(), bins = 10)

In [ ]:
len(data_Opal[(data_Opal['ID'] == b'ee') & (data_Opal['cos_thet']>1)]['cos_thet'].to_numpy())

In [ ]:
len(data_Opal[(data_Opal['ID'] == b'ee')]['cos_thet'].to_numpy())

### Correct background and efficency 

* $N_{cut corr} = N_{cut} \epsilon^{-1}$

To get the error on the background and efficency corrected data one has to propagate the $\sqrt{N}$ error with the error of the inverse efficency matrix calculated using a toy MC and propagate it using gaussian error propagation.
* $\Delta N_{cut corr} = \sqrt{(\Delta N_{cut} \epsilon^{-1})^{2} + (N_{cut} \Delta \epsilon^{-1})^{2}}$

In [ ]:
Matrix = np.array([[9.44358862e-01, 0.00000000e+00, 1.06607535e-03, 2.66518838e-04,
        4.60544551e-03],
       [2.01311705e-05, 9.47722529e-01, 3.42865619e-02, 0.00000000e+00,
        1.79697185e-02],
       [3.82571263e-02, 7.73853107e-03, 9.26237786e-01, 1.13995001e-02,
        1.43535234e-02],
       [9.34935016e-04, 5.07289754e-05, 4.24094234e-03, 9.94561854e-01,
        1.62332721e-04]])


Matrix_inv = np.linalg.inv(Matrix[:, :4])


Matrix_error_inv = np.array([[4.87418322e-03, 9.46085523e-07, 1.15537576e-04, 1.06662703e-04],
                             [3.01977613e-05, 4.92789228e-03, 7.24046709e-04, 1.80927276e-04],
                             [2.31450425e-04, 4.15124017e-05, 2.40414630e-04, 5.00395757e-03],
                             [7.65248259e-04, 3.12274846e-04, 5.14290685e-03, 4.08665348e-04]])[:,:4]

In [ ]:
Ncutcorr = np.zeros((7,4))
Ncutcorr_err = np.zeros((7,4))
Sigma = np.zeros((7,4))
Sigma_err = np.zeros((7,4))
for i in range(0,7,1):
    N_df = pd.DataFrame(classify_Opal_df.loc[classify_Opal_df['CMS_energies'] == meanenergy[i],['es', 'mm', 'tt', 'qq']])
    N_err_df = pd.DataFrame(classify_Opal_df.loc[classify_Opal_df['CMS_energies'] == meanenergy[i],['es_u', 'mm_u', 'tt_u', 'qq_u']])
    N = np.reshape(N_df.to_numpy(dtype=float),4)
    N_err = np.reshape(N_err_df.to_numpy(dtype=float),4)
    Ncor = N.dot(Matrix_inv)
    Nerrcor = np.sqrt( np.square( N.dot(Matrix_error_inv) )+ np.square( N_err.dot(Matrix_inv) ))
    Ncutcorr[i] = Ncor
    Ncutcorr_err[i] = Nerrcor
    sig = Ncor / lumi[i]
    sig_err = np.sqrt( np.square( Nerrcor / lumi[i]) + np.sqrt(Ncor / (lumi[i]**2) * all[i]) )
    Sigma[i] = sig
    Sigma_err[i] = sig_err


### Radiation corrections

In [ ]:
hadronic = [2.0, 4.3, 7.7, 10.8, 4.7, -0.2, -1.6]
leptonic = [0.09, 0.20, 0.36, 0.52, 0.22, -0.01, -0.08]

In [ ]:
for i in range(0,3,1):
    Sigma[:,i] = Sigma[:,i]+leptonic

Sigma[:,3] = Sigma[:,3]+hadronic

In [ ]:
Ncutcorr

In [ ]:
Ncutcorr_err

In [ ]:
Sigma_err

In [ ]:
name = ['electron', 'myon', 'tau', 'hadron']
for i in range(0,4,1):
        plt.errorbar(meanenergy,Sigma[:,i], yerr=Sigma_err[:,i] ,marker ='x', ls= '', ms = 8, capsize=2, capthick=0.5,ecolor='black', elinewidth=0.5, label=f'{name[i]}')

plt.legend()

In [ ]:
name = ['electron', 'myon', 'tau', 'hadron']
for i in range(0,3,1):
        plt.errorbar(meanenergy,Sigma[:,i], yerr=Sigma_err[:,i] ,marker ='x', ls= '', ms = 8, capsize=2, capthick=0.5,ecolor='black', elinewidth=0.5, label=f'{name[i]}')

plt.legend()

In [ ]:
def relativistic_breit_wigner(x, resonance_mass, width, normalization):
    gamma = np.sqrt(resonance_mass ** 2 * (resonance_mass ** 2 + width ** 2))
    k = 2.0 * np.sqrt(2) * resonance_mass * width * gamma / (np.pi * np.sqrt(resonance_mass ** 2 + gamma))
    return normalization * k / ((x ** 2 - resonance_mass ** 2) ** 2 + resonance_mass ** 2 * width ** 2)

MZarray = np.zeros(4)
MZarray_err = np.zeros(4)
widtharray = np.zeros(4)
widtharray_err = np.zeros(4)
crossectionmax = np.zeros(4)
crossectionmax_err = np.zeros(4)
    
for i in range(0,4,1):
    popt, pcov = curve_fit(relativistic_breit_wigner, meanenergy,Sigma[:,i], sigma = Sigma_err[:,i], p0=[90, 10, 30], maxfev = 8000)
    x = np.linspace(80, 100, 200)
    y = relativistic_breit_wigner(x, *popt)
    y_errp = relativistic_breit_wigner(x, popt[0]+np.sqrt(pcov[0][0]), popt[1]+np.sqrt(pcov[1][1]), popt[2]+np.sqrt(pcov[2][2]))
    y_errm = relativistic_breit_wigner(x, popt[0]-np.sqrt(pcov[0][0]), popt[1]-np.sqrt(pcov[1][1]), popt[2]-np.sqrt(pcov[2][2]))
    plt.plot(x, y, label=f'Fit{i} params = {popt[0]:.2f}, {popt[1]:.2f}, {popt[2]:.2f}')
    #plt.plot(x, y_err, label=f'Fit{i} params = {popt[0]:.2f}, {popt[1]:.2f}, {popt[2]:.2f}')
    plt.errorbar(meanenergy,Sigma[:,i], yerr=Sigma_err[:,i] ,marker ='x', ls= '', ms = 8, capsize=2, capthick=0.5,ecolor='black', elinewidth=0.5, label=f'{i}')
    print(popt)
    print()
    print(np.sqrt(pcov))
    print('---------------')
    MZarray[i] = popt[0]
    widtharray[i] =  popt[1]
    crossectionmax[i] = max(y)
    MZarray_err[i] = np.sqrt(pcov[0][0])
    widtharray_err[i] =  np.sqrt(pcov[1][1])
    crossectionmax_err[i] = np.mean([abs(max(y)-max(y_errp)),abs(max(y)-max(y_errm))])
plt.xlabel('CMS energy \GeV')
plt.ylabel('crossection\ nb')
plt.legend()
plt.show()

In [ ]:
def relativistic_breit_wigner(x, resonance_mass, width, normalization):
    gamma = np.sqrt(resonance_mass ** 2 * (resonance_mass ** 2 + width ** 2))
    k = 2.0 * np.sqrt(2) * resonance_mass * width * gamma / (np.pi * np.sqrt(resonance_mass ** 2 + gamma))
    return normalization * k / ((x ** 2 - resonance_mass ** 2) ** 2 + resonance_mass ** 2 * width ** 2)


for i in range(0,3,1):
    popt, pcov = curve_fit(relativistic_breit_wigner, meanenergy,Sigma[:,i], sigma = Sigma_err[:,i], p0=[90, 10, 30], maxfev = 8000)
    x = np.linspace(80, 100, 200)
    y = relativistic_breit_wigner(x, *popt)
    y_err = relativistic_breit_wigner(x, popt[0]+np.sqrt(pcov[0][0]), popt[1]+np.sqrt(pcov[1][1]), popt[2]+np.sqrt(pcov[2][2]))
    plt.plot(x, y, label=f'Fit{i} params = {popt[0]:.2f}, {popt[1]:.2f}, {popt[2]:.2f}')
    #plt.plot(x, y_err, label=f'Fit{i} params = {popt[0]:.2f}, {popt[1]:.2f}, {popt[2]:.2f}')
    plt.errorbar(meanenergy,Sigma[:,i], yerr=Sigma_err[:,i] ,marker ='x', ls= '', ms = 8, capsize=2, capthick=0.5,ecolor='black', elinewidth=0.5, label=f'{i}')
    print(popt)
    print()
    print(np.sqrt(pcov))
    print('---------------')
plt.xlabel('CMS energy \GeV')
plt.ylabel('crossection\ nb')
plt.legend()
plt.show()

## partial decay width

In [ ]:
print(widtharray)
print()
print(MZarray)

In [ ]:
def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- NumPy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    # Fast and numerically precise:
    variance = np.average((values-average)**2, weights=weights)
    return (average, np.sqrt(variance))

In [ ]:
MZ, MZ_err = weighted_avg_and_std(MZarray, MZarray_err)
#MZ_err = np.average(MZarray, weights=MZarray_err)
print(f'the Z0 mass is {MZ} with a stat. uncertainty of {MZ_err}')
width, width_err = weighted_avg_and_std(widtharray, widtharray_err)
#width_err = np.std(widtharray)
print(f'\nthe decay width of Z0 is {width} with a stat. uncertainty of {width_err}')

#print(crossectionmax)   # nB = (h_bar*c/624)^2 * GeV^-2

                        # Mz --> GeV,  width--> Gev
h_bar= 6.582119569*10**(-25) #GeV⋅s
c_light= 299792458 # m/s

Gamma_e = np.sqrt((crossectionmax[0]/624**2) * MZ**2 * width**2 / (12 * np.pi))
Gamma_m = ((crossectionmax[1]/624**2) * MZ**2 * width**2) / (12 * np.pi * Gamma_e)
Gamma_t = ((crossectionmax[2]/624**2) * MZ**2 * width**2) / (12 * np.pi * Gamma_e)
Gamma_q = ((crossectionmax[3]/624**2) * MZ**2 * width**2) / (12 * np.pi * Gamma_e)

Gamma_e_err = np.sqrt( np.square( MZ**2 *width**2 / ( 4672512 * np.pi) * crossectionmax_err[0]) + np.square(crossectionmax[0] * MZ * width**2 / (2336256 * np.pi) * MZ_err) + np.square( crossectionmax[0] * MZ**2 * width / (2336256 *np.pi ) * width_err))

Gamma_m_err = np.sqrt(np.square(MZ**2 * width**2 / (4672512 * np.pi * Gamma_e) * crossectionmax_err[1]) + np.square(crossectionmax[1] * MZ * width**2 / ( 2336256 * np.pi * Gamma_e) * MZ_err) + np.square( crossectionmax[1] * MZ * width / ( 2336256 * np.pi * Gamma_e) * width_err)+ np.square(crossectionmax[1] * MZ**2 * width**2 / (4672512 * np.pi * Gamma_e**2) * Gamma_e_err))
Gamma_t_err = np.sqrt(np.square(MZ**2 * width**2 / (4672512 * np.pi * Gamma_e) * crossectionmax_err[2]) + np.square(crossectionmax[2] * MZ * width**2 / ( 2336256 * np.pi * Gamma_e) * MZ_err) + np.square( crossectionmax[2] * MZ * width / ( 2336256 * np.pi * Gamma_e) * width_err)+ np.square(crossectionmax[2] * MZ**2 * width**2 / (4672512 * np.pi * Gamma_e**2) * Gamma_e_err))
Gamma_q_err = np.sqrt(np.square(MZ**2 * width**2 / (4672512 * np.pi * Gamma_e) * crossectionmax_err[3]) + np.square(crossectionmax[3] * MZ * width**2 / ( 2336256 * np.pi * Gamma_e) * MZ_err) + np.square( crossectionmax[3] * MZ * width / ( 2336256 * np.pi * Gamma_e) * width_err)+ np.square(crossectionmax[3] * MZ**2 * width**2 / (4672512 * np.pi * Gamma_e**2) * Gamma_e_err))

Gamma_v = width - Gamma_e - Gamma_m - Gamma_q - Gamma_t
Gamma_v_err = np.sqrt( Gamma_e_err**2 + Gamma_m_err**2 + Gamma_t_err**2 + Gamma_q_err**2 + width_err**2)
print(f'\nPartial decay widths with stat uncertainty \nGamma_e = {Gamma_e:.4f} \pm {Gamma_e_err:.4f}; Gamma_m = {Gamma_m:.4f} \pm {Gamma_m_err:.4f}; Gamma_t = {Gamma_t:.4f} \pm {Gamma_t_err:.4f}; Gamma_q = {Gamma_q:.4f} \pm {Gamma_q_err:.4f}; Gamma_v = {Gamma_v:.4f} \pm {Gamma_v_err:.4f}')
Gamma = np.array([Gamma_e, Gamma_m, Gamma_t, Gamma_q, Gamma_v])

In [ ]:
crossectionmax

## 3.2 Backward-Forward-Symmetry $A_{FB}$ ##

The $A_{FB} can be calculated by equation (19) of the english instuctions. To caluclate it with the methods we have, first eq(19) is simplified to
\begin{align}
A_{FB}= \frac{\sigma_B-\sigma_F}{\sigma_B+\sigma_F}
\end{align}

with 

\begin{align}
\sigma_{F,B} = \frac{N_{F, B}}{\int L dt}
\end{align}
The numbers $N_{F,B}$ can be extracted from the myon plot of $\cos_{thet}$ with conditions $\lessgtr cos(\theta)=0$, respectively.
Therefore $A_{FB}$ becomes
\begin{align}
A_{FB}= \frac{N_B-N_F}{N_B+N_F}
\end{align}

errors are calculated with gaussian error propagation, see 2nd code space after this markdown. Talks about the dependecies of the the symmetries with the COM-energy will be in the report.

In [ ]:
#meanenergy = np.array([88.47939, 89.46793, 90.22266, 91.22430, 91.96648, 92.96465, 93.71712])

A_FB= np.zeros(7)
N_F_array = np.zeros(7)
N_B_array = np.zeros(7)

for i in range(0,7,1):

    myon_cos= data_Opal[(data_Opal['ID']== b'mm') & (data_Opal['cos_thet']<200) & (data_Opal['COM_energy']==meanenergy[i])]["cos_thet"].to_numpy()
    myon_cos_F= data_Opal[(data_Opal['ID']== b'mm') & (data_Opal['cos_thet']<0)& (data_Opal['COM_energy']==meanenergy[i])]["cos_thet"].to_numpy()
    myon_cos_B= data_Opal[(data_Opal['ID']== b'mm') & (data_Opal['cos_thet']<200)& (data_Opal['cos_thet']>0)& (data_Opal['COM_energy']==meanenergy[i])]["cos_thet"].to_numpy()

    N_ges= len(myon_cos)
    N_F= len(myon_cos_F)
    N_B= len(myon_cos_B)
    N_F_array[i] = N_F
    N_B_array[i] = N_B
    A_FB[i] = (N_B-N_F)/N_ges

A_FB_correction= np.array([0.021512, 0.019262, 0.016713, 0.018293, 0.030286, 0.062196, 0.093850])

A_FB= A_FB+A_FB_correction

#plt.hist(myon_cos_F, bins= 100)
#plt.hist(myon_cos_B, bins= 100)
#plt.show()


#plt.plot(meanenergy, A_FB, '.')

print('\n')
print('A_FB=', A_FB)

#plt.grid()
#plt.show()

In [ ]:
Nf = N_F_array[3]
Nb = N_B_array[3]
VdA = np.sqrt(A_FB[3]/3)
sin2thetaWein = (1-VdA)/4
Delta_sin2thetaWein = np.sqrt( np.square( (2*Nf) / (Nb+Nf)**2 * np.sqrt(Nb)) + np.square( (2*Nb) / (Nb+Nf)**2 * np.sqrt(Nb)))
print(f'{sin2thetaWein} $\pm$ {Delta_sin2thetaWein}')

## 4. Lepton universality ##

The Lepton universality states, that every decay from particles with high centre-of-mass energy into leptons of any family has the same probability, or stating it in other terms, posses the same crosssection and partial width. For a more convenient comparison, the crossections of the leptons are used, because they are calculated earlier and are thus less frail against systematic errors, e.g. all partial widths depend heavily on the good reproduction of the partial width $\Gamma_e$ and so on.

To compare the crossections, all three values are compared pairwise via the t-test

\begin{align}
t-test= \frac{|a-b|}{\sqrt{\sigma_a^2 + \sigma_b^2}}
\end{align}

The two values are comparable, if the value of the t-test is $\in [0,2]$.

def t_test(a,sig_a, b, sig_b):
    return np.abs(a-b)/np.sqrt(sig_a**2+sig_b**2)

print(crossectionmax[:3])
crossectionmax_err= np.array([0.03321685, 0.01366334, 0.01515388, 0.13298636]) # ee, mm, tt, qq
print(crossectionmax_err[:3])

ttest= np.zeros(3)

for i in range(3):
    #print(i%3, (i+1)%3)
    ttest[i]= t_test(crossectionmax[i%3],crossectionmax_err[i%3], crossectionmax[(i+1)%3], crossectionmax_err[(i+1)%3])

print(ttest, '--> only first pair is comparable!')


Further the ratios of the Hadron peak crossection with the lepton crossections and its the branching ratios should be calculated.

Error propagation is just normal gaussian error propagation from used values, which errors are derived above. Due to time, the exact calculation will just be in the report.

In [ ]:
cs_ratio= np.zeros(3)
branching_ratio= np.zeros(5)

for i in range(3):
    cs_ratio[i]= crossectionmax[i]/crossectionmax[3]

for i in range(5):
    branching_ratio[i]= Gamma[i]/width

print('Crossection ratio      =', cs_ratio, ',  ee/qq, mm/qq, tt/qq')
print('partial crossections   =', Gamma, ',  ee, mm, tt, qq, vv(neutrinos)')
print('branching ratio        =', branching_ratio, ',  ee, mm, tt, qq, vv(neutrinos)')
print('sum of branching ratios=', sum(branching_ratio))